In [1]:
'''
Title: Batter Performance Table Pt 2
Author: Sam LaFell
Date: 11/5/2022
Purpose: Part 1 didnt seem to work

Granularity: xBA by pitch for each batter. By pitcher handiness.
'''

# Import Libraries
import pandas as pd
pd.set_option('display.max_columns', 500)
import pybaseball as pb
import os
import numpy as np
import re
from datetime import datetime, timedelta

In [2]:
import sys
import pathlib
import os 

# Set base path
base_path = os.path.abspath('/Users/samlafell/Desktop/sports_betting_project')
os.chdir(base_path)

# Add utils folder if not already in system path
if os.path.join(base_path, 'utils') not in sys.path:
    print('adding path')
    sys.path.append(os.path.join(base_path, 'utils'))
    
from utils import get_players, game_performances, plate_appearances, expected_ba

adding path


## Lookup the Player IDs for all players on a specific Team
## Then use this to get their performance information for games in a time period leading up to a certain date

In [3]:
# Generate a lookup table to use in the Game Performances function
atl_players = get_players.get_players('2018-05-13', 'ATL', min_abs_season=10)

Gathering player lookup table. This may take a moment.


In [4]:
atl_players

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last,playerID,POS
0,freeman,freddie,518692,freef001,freemfr01,5361,2010.0,2022.0,freemfr01,1B
1,acuna,ronald,660670,acunr001,acunaro01,18401,2018.0,2022.0,acunaro01,OF
2,albies,ozzie,645277,albio001,albieoz01,16556,2017.0,2022.0,albieoz01,2B
3,inciarte,ender,542255,incie001,inciaen01,4922,2014.0,2022.0,inciaen01,OF
4,camargo,johan,622666,camaj001,camarjo01,14950,2017.0,2022.0,camarjo01,3B
5,markakis,nick,455976,markn001,markani01,5930,2006.0,2020.0,markani01,OF
6,flowers,tyler,452095,flowt001,flowety01,9134,2009.0,2020.0,flowety01,C
7,swanson,dansby,621020,swand001,swansda01,18314,2016.0,2022.0,swansda01,SS
8,culberson,charlie,518586,culbc001,culbech01,3298,2012.0,2022.0,culbech01,SS
9,suzuki,kurt,435559,suzuk001,suzukku01,8259,2007.0,2022.0,suzukku01,C


In [5]:
# If Game Date is after 5/1, do this, else we will need previous season numbers
game_performances_df = game_performances.game_performances('2018-05-13', atl_players, lookback_period=28)

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data


In [7]:
# Get plate appaearnce info in this time
atbats = plate_appearances.plate_appearances(game_performances_df, min_pas = 15, p_handiness=False)

# Get Expected BA Count
expected_ba_df = expected_ba.expected_ba(game_performances_df, atbats.batter.values, p_handiness=False)

# Batters PAs and xBA to date
pas_xba = atbats.merge(expected_ba_df, how='inner', on='batter').rename(columns={'estimated_ba_using_speedangle':'xBA'})
pas_xba.sort_values('xBA', ascending=False)

,batter,PA,xBA
5,518692,106,0.420241
2,455976,108,0.380409
1,452095,25,0.370882
4,518586,22,0.361462
7,621020,62,0.345474
10,660670,72,0.344102
6,542255,110,0.335621
9,645277,113,0.307402
3,475247,62,0.303538
8,622666,56,0.301314


# Conclusion
Now, I have batter performance in the last x days from the current day game for a team, let's move to Pitching performances